<a href="https://colab.research.google.com/github/mannam95/Deep_Learning_Programming/blob/main/MicroExpSTCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import statements

In [ ]:
import os
import cv2
import numpy
import imageio
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
# from keras.optimizers import SGD, RMSprop
from tensorflow.keras.optimizers import SGD, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from keras import backend as K
# import tensorflow.keras.backend as K
import sys

In [ ]:
# K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')

# Change the directory

In [ ]:
working_directory = '/content/drive/My Drive/Colab Notebooks/Kranthi_Task/'
def colabDrive():
    from google.colab import drive
    drive.mount("/content/drive")

    if os.getcwd() !=  working_directory:
      os.chdir(working_directory)
    print(os.getcwd())

In [ ]:
colabDrive()

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Kranthi_Task


In [ ]:
# !unzip 'HS.zip' -d 'Temp'

# Old Code

In [ ]:
image_rows, image_columns, image_depth = 64, 64, 18
training_list = []
subjectPath = 'Temp/HS/'

In [ ]:
subDirectory = os.listdir(subjectPath)
for sub in subDirectory:
  negativepath = 'Temp/HS/'+ sub +'/micro/negative/'
  directorylisting = os.listdir(negativepath)
  for video in directorylisting:
      videopath = negativepath + video
      frames = []
      framelisting = os.listdir(videopath)
      if len(framelisting) > 18:
        framerange = [x for x in range(18)]
        for frame in framerange:
              imagepath = videopath + "/" + framelisting[frame]
              image = cv2.imread(imagepath)
              imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
              grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
              frames.append(grayimage)
        frames = numpy.asarray(frames)
        videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
        training_list.append(videoarray)

In [ ]:
subDirectory = os.listdir(subjectPath)
for sub in subDirectory:
  positivepath = 'Temp/HS/'+ sub +'/micro/positive/'
  directorylisting = os.listdir(positivepath)
  for video in directorylisting:
      videopath = positivepath + video
      frames = []
      framelisting = os.listdir(videopath)
      if len(framelisting) > 18:
        framerange = [x for x in range(18)]
        for frame in framerange:
              imagepath = videopath + "/" + framelisting[frame]
              image = cv2.imread(imagepath)
              imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
              grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
              frames.append(grayimage)
        frames = numpy.asarray(frames)
        videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
        training_list.append(videoarray)

In [ ]:
subDirectory = os.listdir(subjectPath)
for sub in subDirectory:
  surprisepath = 'Temp/HS/'+ sub +'/micro/surprise/'
  directorylisting = os.listdir(surprisepath)
  for video in directorylisting:
      videopath = surprisepath + video
      frames = []
      framelisting = os.listdir(videopath)
      if len(framelisting) > 18:
        framerange = [x for x in range(18)]
        for frame in framerange:
              imagepath = videopath + "/" + framelisting[frame]
              image = cv2.imread(imagepath)
              imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
              grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
              frames.append(grayimage)
        frames = numpy.asarray(frames)
        videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
        training_list.append(videoarray)

# Pre-Process

In [ ]:
image_rows, image_columns, image_depth = 64, 64, 18
training_list = []
subjectPath = 'Temp/HS/'

## Read Data

In [ ]:
for lab in ['negative', 'positive', 'surprise']:
  subDirectory = os.listdir(subjectPath)
  for sub in subDirectory:
    labelpath = 'Temp/HS/'+ sub +'/micro/' + lab + '/'
    directorylisting = os.listdir(labelpath)
    for video in directorylisting:
        videopath = labelpath + video
        frames = []
        framelisting = os.listdir(videopath)
        if len(framelisting) > 18:
          framerange = [x for x in range(18)]
          for frame in framerange:
                imagepath = videopath + "/" + framelisting[frame]
                image = cv2.imread(imagepath)
                imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
                grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
                frames.append(grayimage)
          frames = numpy.asarray(frames)
          videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
          training_list.append(videoarray)

## Process Data into numpy arrays

In [ ]:
training_list = numpy.asarray(training_list)
trainingsamples = len(training_list)
traininglabels = numpy.zeros((trainingsamples, ), dtype = int)

# Assign Labels
traininglabels[0:66] = 0
traininglabels[66:116] = 1
traininglabels[113:159] = 2

# Softmax
traininglabels = np_utils.to_categorical(traininglabels, 3)

In [ ]:
training_data = [training_list, traininglabels]
(trainingframes, traininglabels) = (training_data[0], training_data[1])
training_set = numpy.zeros((trainingsamples, 1, image_rows, image_columns, image_depth))

for h in range(trainingsamples):
    training_set[h][0][:][:][:] = trainingframes[h, :, :, :]

training_set = training_set.astype('float32')
training_set -= numpy.mean(training_set)
training_set /= numpy.max(training_set)

In [ ]:
# Save training images and labels in a numpy array
numpy.save('numpy_training_datasets/microexpstcnn_images.npy', training_set)
numpy.save('numpy_training_datasets/microexpstcnn_labels.npy', traininglabels)

In [ ]:
#Load the saved numpy data
training_set = numpy.load('numpy_training_datasets/microexpstcnn_images.npy')
traininglabels =numpy.load('numpy_training_datasets/microexpstcnn_labels.npy')

In [ ]:
# MicroExpSTCNN Model
model = Sequential()
model.add(Convolution3D(32, (3, 3, 15), input_shape=(1, image_rows, image_columns, image_depth), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, kernel_initializer='normal'))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 32, 62, 62, 4)     4352      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 32, 20, 20, 1)    0         
 3D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 32, 20, 20, 1)     0         
                                                                 
 flatten_4 (Flatten)         (None, 12800)             0         
                                                                 
 dense_8 (Dense)             (None, 128)               1638528   
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                      

In [ ]:
# Spliting the dataset into training and validation sets
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_set, traininglabels, test_size=0.2, random_state=4)

In [ ]:
numpy.save('numpy_validation_dataset/microexpstcnn_val_images.npy', validation_images)
numpy.save('numpy_validation_dataset/microexpstcnn_val_labels.npy', validation_labels)

In [ ]:
validation_images = numpy.load('numpy_validation_dataset/microexpstcnn_val_images.npy')
validation_labels = numpy.load('numpy_validation_dataset/microexpstcnn_val_labels.npy')

In [ ]:
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), batch_size = 8, epochs = 100, shuffle=True)


Epoch 1/100
16/16 [==============================] - 1s 19ms/step - loss: 1.1075 - accuracy: 0.3622 - val_loss: 1.0967 - val_accuracy: 0.3750
Epoch 2/100
16/16 [==============================] - 0s 8ms/step - loss: 1.1075 - accuracy: 0.3701 - val_loss: 1.0503 - val_accuracy: 0.4688
Epoch 3/100
16/16 [==============================] - 0s 8ms/step - loss: 1.0944 - accuracy: 0.3465 - val_loss: 1.0488 - val_accuracy: 0.5312
Epoch 4/100
16/16 [==============================] - 0s 8ms/step - loss: 1.0638 - accuracy: 0.4409 - val_loss: 1.0562 - val_accuracy: 0.4688
Epoch 5/100
16/16 [==============================] - 0s 8ms/step - loss: 1.0747 - accuracy: 0.4409 - val_loss: 0.9982 - val_accuracy: 0.6562
Epoch 6/100
16/16 [==============================] - 0s 8ms/step - loss: 1.1015 - accuracy: 0.3937 - val_loss: 1.0202 - val_accuracy: 0.5625
Epoch 7/100
16/16 [==============================] - 0s 8ms/step - loss: 1.0700 - accuracy: 0.4252 - val_loss: 1.0479 - val_accuracy: 0.4375
Epoch 8/100


In [ ]:
predictions = model.predict(validation_images)
predictions_labels = numpy.argmax(predictions, axis=1)
validation_labels = numpy.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)

[[13  4  4]
 [ 1  1  1]
 [ 3  3  2]]
